In [10]:
import pickle
import numpy as np
from collections import defaultdict

# --- USER SETTING ---
PERCENTILE = 75  # flag all windows above this percentile (e.g. 95th percentile)

# --- Load Results ---
with open('ensemble_res_500.pkl', 'rb') as f:
    all_results = pickle.load(f)

# --- Compute Anomaly Scores ---
anomaly_scores = defaultdict(lambda: {'score_sum': 0.0, 'count': 0})

for iteration_result in all_results:
    buckets = iteration_result['buckets']
    bucket_results = iteration_result['bucket_results']

    for bucket_result in bucket_results:
        bucket_idx = bucket_result['bucket_idx']
        final_results = bucket_result['final_results']
        indices_in_bucket = buckets[bucket_idx]

        mean = np.mean(final_results)
        std = np.std(final_results) if np.std(final_results) != 0 else 1e-8

        for i, idx in enumerate(indices_in_bucket):
            sim = final_results[i]
            deviation = abs(sim - mean) / std
            anomaly_scores[idx]['score_sum'] += deviation
            anomaly_scores[idx]['count'] += 1

# --- Final Score Calculation ---
final_scores = {
    idx: score_data['score_sum'] / score_data['count']
    for idx, score_data in anomaly_scores.items()
}

# --- Compute Threshold by Percentile ---
all_score_values = list(final_scores.values())
threshold = np.percentile(all_score_values, PERCENTILE)

# --- Flag Anomalous Windows ---
anomalous_windows = {
    idx: score for idx, score in final_scores.items() if score >= threshold
}

print(f"\n{len(anomalous_windows)} anomalous windows detected above the {PERCENTILE}th percentile (threshold ≈ {threshold:.4f}):\n")
for idx, score in sorted(anomalous_windows.items()):
    print(f"Window {idx} - Score: {score:.4f}")

# --- Save to file ---
with open("flagged_anomalous_windows500.txt", "w") as f:
    for idx, score in sorted(anomalous_windows.items()):
        f.write(f"Window {idx} - Score: {score:.4f}\n")

print("\nSaved to 'flagged_anomalous_windows.txt'")



40 anomalous windows detected above the 75th percentile (threshold ≈ 0.8648):

Window 8 - Score: 0.9123
Window 9 - Score: 0.8812
Window 10 - Score: 0.9125
Window 11 - Score: 0.8697
Window 12 - Score: 0.8765
Window 27 - Score: 0.8650
Window 107 - Score: 0.8702
Window 108 - Score: 0.8920
Window 109 - Score: 0.9675
Window 110 - Score: 0.9416
Window 111 - Score: 1.0093
Window 112 - Score: 0.9952
Window 113 - Score: 0.9076
Window 114 - Score: 0.8943
Window 115 - Score: 0.9255
Window 116 - Score: 0.8922
Window 117 - Score: 0.9029
Window 118 - Score: 0.9174
Window 119 - Score: 0.9009
Window 120 - Score: 0.8867
Window 121 - Score: 0.8802
Window 122 - Score: 0.9504
Window 123 - Score: 0.8767
Window 124 - Score: 0.9128
Window 126 - Score: 0.8775
Window 127 - Score: 0.8755
Window 128 - Score: 0.8758
Window 130 - Score: 0.9197
Window 131 - Score: 0.8849
Window 132 - Score: 0.9580
Window 133 - Score: 0.8894
Window 134 - Score: 0.9459
Window 135 - Score: 0.9241
Window 136 - Score: 0.9330
Window 137